# MLPRouter - Inference

This notebook demonstrates how to use a trained **MLPRouter** for inference.

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)

In [ ]:
from llmrouter.models.mlprouter import MLPRouter
from llmrouter.utils import setup_environment, load_model, get_longformer_embedding
import yaml

setup_environment()

## 2. Load Trained Router

In [ ]:
CONFIG_PATH = "configs/model_config_train/mlprouter.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

config['model_path']['load_model_path'] = config['model_path']['save_model_path']

INFERENCE_CONFIG_PATH = "configs/model_config_test/mlprouter_inference.yaml"
os.makedirs(os.path.dirname(INFERENCE_CONFIG_PATH), exist_ok=True)

with open(INFERENCE_CONFIG_PATH, 'w') as f:
    yaml.dump(config, f)

router = MLPRouter(yaml_path=INFERENCE_CONFIG_PATH)
print(f"Router loaded with {len(router.llm_data)} LLM candidates")

## 3. Single Query Routing

In [ ]:
EXAMPLE_QUERIES = [
    {"query": "What is the capital of France?"},
    {"query": "Solve the equation: 2x + 5 = 15"},
    {"query": "Write a Python function to check if a number is prime."},
    {"query": "Explain quantum computing in simple terms."},
]

print("Routing Results:")
print("=" * 60)

for i, query in enumerate(EXAMPLE_QUERIES, 1):
    result = router.route_single(query)
    print(f"{i}. {query['query'][:50]}...")
    print(f"   Routed to: {result['model_name']}")

## 4. Confidence Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

model_path = os.path.join(PROJECT_ROOT, config['model_path']['load_model_path'])
mlp_model = load_model(model_path)

def get_confidence(query_text):
    embedding = get_longformer_embedding(query_text).numpy().reshape(1, -1)
    proba = mlp_model.predict_proba(embedding)[0]
    prediction = mlp_model.predict(embedding)[0]
    confidence = max(proba)
    return prediction, confidence, dict(zip(mlp_model.classes_, proba))

print("Confidence Analysis:")
print("=" * 60)
for query in EXAMPLE_QUERIES:
    pred, conf, probs = get_confidence(query['query'])
    print(f"Query: {query['query'][:40]}...")
    print(f"  Prediction: {pred} (Confidence: {conf:.4f})")

## Summary

This notebook demonstrated:
1. Loading a trained MLPRouter
2. Single query routing
3. Confidence analysis

MLPRouter is effective for:
- Complex decision boundaries
- Large-scale routing with many LLMs